<a href="https://colab.research.google.com/github/gitleon8301/MY-AI-Gizmo-working/blob/main/Colab-TextGen-GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# oobabooga/text-generation-webui

After running both cells, a public gradio URL will appear at the bottom in around 10 minutes. You can optionally generate an API link.

* Project page: https://github.com/oobabooga/text-generation-webui
* Gradio server status: https://status.gradio.app/

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
# ================================================================
# MY-AI-Gizmo • LAUNCHER WITH VERBOSE INSTALLATION
# Shows exactly what's happening during installation
# ================================================================

import os
import subprocess
import shutil
import re
import time
from pathlib import Path

try:
    from google.colab import drive
except:
    drive = None

# Configuration
REPO_ZIP = "https://github.com/gitleon8301/MY-AI-Gizmo-working/archive/refs/heads/main.zip"
WORK_DIR = Path("/content/text-generation-webui")
DRIVE_ROOT = Path("/content/drive/MyDrive/MY-AI-Gizmo")

def sh(cmd, check=False):
    return subprocess.run(cmd, shell=True, capture_output=True, text=True, check=check)

def sh_live(cmd):
    """Run command and show output in real-time"""
    proc = subprocess.Popen(
        cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
        text=True, bufsize=1
    )
    for line in proc.stdout:
        print(line, end='')
    proc.wait()
    return proc.returncode

print("🔧 Setting up environment...")
if "MPLBACKEND" in os.environ:
    del os.environ["MPLBACKEND"]
os.environ["MPLBACKEND"] = "Agg"
print("✓ Environment ready\n")

# ═══════════════════════════════════════════════════════════════
# CUDA FIX
# ═══════════════════════════════════════════════════════════════

def fix_cuda_library_path():
    print("🔧 Fixing CUDA library paths...")
    cuda_paths = [
        '/usr/local/cuda/lib64',
        '/usr/local/cuda-12/lib64',
        '/usr/lib/x86_64-linux-gnu',
        '/usr/local/nvidia/lib64',
    ]
    valid_paths = []
    for path in cuda_paths:
        if Path(path).exists() and list(Path(path).glob('libcuda.so*')):
            valid_paths.append(path)
            print(f"  ✓ {path}")
    if valid_paths:
        os.environ['LD_LIBRARY_PATH'] = ':'.join(valid_paths)
        print(f"  ✓ Set LD_LIBRARY_PATH")
        return True
    return False

# ═══════════════════════════════════════════════════════════════
# FILE FUNCTIONS
# ═══════════════════════════════════════════════════════════════

def _ensure_drive_path(drive_path: Path, is_settings_file=False):
    if drive_path.suffix:
        drive_path.parent.mkdir(parents=True, exist_ok=True)
        if is_settings_file and not drive_path.exists():
            drive_path.touch(exist_ok=True)
    else:
        drive_path.mkdir(parents=True, exist_ok=True)

def _remove_path(path: Path):
    try:
        if path.is_symlink():
            path.unlink()
        elif path.is_dir():
            shutil.rmtree(path)
        elif path.exists():
            path.unlink()
    except:
        pass

def _create_symlink_or_fallback(src: Path, dest: Path):
    try:
        dest.parent.mkdir(parents=True, exist_ok=True)
        if dest.exists() or dest.is_symlink():
            _remove_path(dest)
        os.symlink(str(src), str(dest), target_is_directory=src.is_dir())
        return True
    except:
        try:
            if src.is_dir():
                if dest.exists():
                    _remove_path(dest)
                shutil.copytree(src, dest)
            else:
                dest.parent.mkdir(parents=True, exist_ok=True)
                shutil.copy2(src, dest)
            return True
        except:
            return False

def cleanup_broken_files(drive_root: Path):
    print("\n🧹 Cleaning broken files...")
    models_dir = drive_root / "models"
    if not models_dir.exists():
        print("  No models directory yet")
        return
    extensions = ["*.gguf", "*.safetensors", "*.bin", "*.pth", "*.pt"]
    broken = []
    for ext in extensions:
        for f in models_dir.rglob(ext):
            if f.stat().st_size < (100 * 1024):
                broken.append(f)
    if broken:
        print(f"  Found {len(broken)} broken files - deleting...")
        for f in broken:
            try:
                f.unlink()
            except:
                pass
        print(f"  ✓ Cleaned")
    else:
        print("  ✓ No broken files")

# ═══════════════════════════════════════════════════════════════
# MAIN SETUP
# ═══════════════════════════════════════════════════════════════

print("=" * 60)
print("🚀 MY-AI-Gizmo Setup (Verbose Mode)")
print("=" * 60)

# Step 1: Mount
print("\n📁 Step 1/6: Mounting Drive...")
if drive:
    try:
        drive.mount("/content/drive", force_remount=False)
        print("✓ Mounted")
    except Exception as e:
        print(f"⚠️  {e}")

# Cleanup
cleanup_broken_files(DRIVE_ROOT)

# Step 2: Create folders
print("\n💾 Step 2/6: Creating folders...")
folders = ["models", "loras", "training", "characters", "presets", "prompts",
           "settings", "chat-history", "instruct-history", "outputs", "images",
           "logs", "cache", "extensions", "softprompts"]
for f in folders:
    (DRIVE_ROOT / f).mkdir(parents=True, exist_ok=True)
print(f"✓ {len(folders)} folders")

# Step 3: Download repo
print("\n📥 Step 3/6: Repository...")
if not WORK_DIR.exists():
    sh("rm -f /content/repo.zip")
    print("  Downloading...")
    sh(f"wget -q -O /content/repo.zip {REPO_ZIP}")
    sh("unzip -q /content/repo.zip -d /content")
    try:
        next(Path("/content").glob("MY-AI-Gizmo-working-*")).rename(WORK_DIR)
        print("✓ Downloaded")
    except:
        print("⚠️  Failed")
else:
    print("✓ Exists")

if WORK_DIR.exists():
    os.chdir(WORK_DIR)

# Step 4: Symlinks
print("\n🔗 Step 4/6: Linking...")
links_map = [
    ("models", "models", False),
    ("loras", "loras", False),
    ("user_data/characters", "characters", False),
    ("user_data/presets", "presets", False),
    ("user_data/settings.yaml", "settings/settings.yaml", True),
    ("user_data/settings.json", "settings/settings.json", True),
    ("user_data/chat", "chat-history", False),
    ("outputs", "outputs", False),
]

for local, drive_folder, is_settings in links_map:
    drive_path = DRIVE_ROOT / drive_folder
    _ensure_drive_path(drive_path, is_settings_file=is_settings)
    local_path = WORK_DIR / local
    if local_path.exists() or local_path.is_symlink():
        _remove_path(local_path)
    local_path.parent.mkdir(parents=True, exist_ok=True)
    _create_symlink_or_fallback(drive_path, local_path)
print("✓ Linked")

# Step 5: Settings
print("\n⚙️  Step 5/6: Settings...")
drive_settings = DRIVE_ROOT / "settings" / "settings.yaml"
local_settings = WORK_DIR / "user_data" / "settings.yaml"
local_settings.parent.mkdir(parents=True, exist_ok=True)
if local_settings.is_symlink():
    local_settings.unlink()
if drive_settings.exists() and drive_settings.stat().st_size > 0:
    shutil.copy2(drive_settings, local_settings)
    print("✓ Copied from Drive")
else:
    local_settings.write_text("# minimal\nlisten: true\nshare: true\n")
    print("✓ Created")

# ═══════════════════════════════════════════════════════════════
# Step 6: Install (WITH LIVE OUTPUT)
# ═══════════════════════════════════════════════════════════════

print("\n📦 Step 6/6: Installing dependencies...")
print("=" * 60)
print("📊 LIVE INSTALLATION OUTPUT (showing progress)")
print("=" * 60)
print("\nThis may take 10-15 minutes on first run...")
print("Watch for these stages:")
print("  1️⃣  Creating environment")
print("  2️⃣  Installing PyTorch (~2GB)")
print("  3️⃣  Installing packages")
print("  4️⃣  Compiling llama-cpp (slowest part)")
print("  5️⃣  Finishing up")
print("\n" + "=" * 60 + "\n")

sh("chmod +x start_linux.sh")

# Run with LIVE output
start_time = time.time()
returncode = sh_live("GPU_CHOICE=A LAUNCH_AFTER_INSTALL=FALSE INSTALL_EXTENSIONS=FALSE bash start_linux.sh")
elapsed = time.time() - start_time

print("\n" + "=" * 60)
if returncode == 0:
    print(f"✅ Installation complete ({elapsed:.1f}s)")
else:
    print(f"⚠️  Installation warnings (took {elapsed:.1f}s)")
print("=" * 60)

# ═══════════════════════════════════════════════════════════════
# CUDA FIX
# ═══════════════════════════════════════════════════════════════

print("\n🔧 Setting up CUDA...")
fix_cuda_library_path()

# ═══════════════════════════════════════════════════════════════
# LAUNCH
# ═══════════════════════════════════════════════════════════════

print("\n" + "=" * 70)
print("🌐 LAUNCHING UI")
print("=" * 70 + "\n")

sh("pkill -9 -f python")
time.sleep(2)

env = os.environ.copy()
env["MPLBACKEND"] = "Agg"
if "PYTHONPATH" in env:
    del env["PYTHONPATH"]

python_exe = str(WORK_DIR / "installer_files/env/bin/python")
if not Path(python_exe).exists():
    python_exe = "python3"

cmd = f"{python_exe} -u server.py --share --listen"

proc = subprocess.Popen(
    cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
    text=True, bufsize=1, env=env, cwd=str(WORK_DIR)
)

shown = False
for line in proc.stdout:
    print(line, end="")

    if not shown and ("Running on public URL:" in line or "Running on local URL:" in line):
        if (m := re.search(r"(https://[a-z0-9]+\.gradio\.live)", line)):
            print("\n" + "=" * 70)
            print("✨ SUCCESS! ✨")
            print("=" * 70)
            print(f"\n🌍 PUBLIC: {m.group(1)} ← CLICK!")
            print("\n🎯 Model tab → Select model → Load → Chat!")
            print("=" * 70 + "\n")
            shown = True

print("\n✓ Done")

✅ RECOMMENDED MODELS (COPY EXACTLY)
🔹 BEST GENERAL CHAT (START HERE)

Llama-2-7B-Chat

Repo: TheBloke/Llama-2-7B-Chat-GGUF
File: llama-2-7b-chat.Q4_K_M.gguf

🔹 FAST + LIGHT (LOW RAM)

TinyLlama-1.1B-Chat

Repo: TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF
File: tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf

🔹 STRONG CHAT (BETTER THAN LLAMA-2)

Mistral-7B-Instruct

Repo: TheBloke/Mistral-7B-Instruct-v0.2-GGUF
File: mistral-7b-instruct-v0.2.Q4_K_M.gguf

🔹 CODING MODEL

Code LLaMA-7B

Repo: TheBloke/CodeLlama-7B-GGUF
File: codellama-7b.Q4_K_M.gguf

🔹 ROLEPLAY / STORY

MythoMax-L2-13B (needs more RAM)

Repo: TheBloke/MythoMax-L2-13B-GGUF
File: mythomax-l2-13b.Q4_K_M.gguf

🔹 VERY FAST / TEST MODEL

Phi-2 (2.7B)

Repo: TheBloke/phi-2-GGUF
File: phi-2.Q4_K_M.gguf

⚙️ WHAT LOADER TO USE (IMPORTANT)

For ALL models above:

Loader: llama.cpp


Repo: TheBloke/Llama-2-7B-Chat-GGUF
File: llama-2-7b-chat.Q4_K_M.gguf
